In [1]:
import matplotlib.pyplot as plt
import math
import random
from qiskit import IBMQ
from math import sqrt
# importing Qiskit
from qiskit.utils import QuantumInstance, algorithm_globals
# importing backend 
from typing import List
from qiskit.utils import QuantumInstance
# importing Grover 
from GroverFSP import GroverFlowSop
import numpy as np

In [2]:
class GroverOptimizerFSP :
    def __init__(
        self,
        num_jobs : int,
        num_machines : int,
        process_time : List[List[int]],
        upper : int,
        quantum_instance : QuantumInstance,
        num_iteration  
        )-> None :
        """
            Args : 
            problem : flow shop instance {m,n,processsiog time matrix}
            quantum instance : backend to execute the circuit
            num_iteration : number of iteration if there is no improvement
        """
        self.num_jobs = num_jobs
        self.num_machines = num_machines
        self.process_time = process_time
        self.upper = upper
        self.quantum_instance = quantum_instance
        self.num_iteration = num_iteration

    def solve(self):
        """
            GAS solver 
        """
        # Optimum tracking variables
        optimum_found = False
        optimum_permu = math.inf
        optimum_value = math.inf
        trace = []
        # Grover ciruit parameters
        upperBound = self.upper
        n = self.num_jobs
        N = 2**n
        m = self.num_machines
        q = self.qubits_cmj_Estimation()
        pm = self.process_time       
        quantum_instance = self.quantum_instance
        # solotions tracking
        num_solutions = 2**(N*n)
        schedule_measurement = []
        # Grover result
        grover_output = {}
        # Stop the algorithm if we hit the rotation max
        r = 0 
        max_r = int(math.ceil(100 * math.pi / 4))
        algo_tracking = []
        it = 0
        while not optimum_found :
            m = 1
            impovement = False
            it+=1
            print(it)
            # Iterate until we don't get improvement
            nb_no_improvement = 0
            while not impovement :
                # The required number of rotation 
                nb_no_improvement += 1
                r_count = random.randint(1,m)
                r += r_count
                print("startGrover")
                # Create Grover FSP Circuit
                grover = GroverFlowSop(n,q,pm,m,upperBound + 1,r_count,quantum_instance)
                # Execute Grover 
                grover_output = grover.execute()
                print(grover_output)
                # Choose a random solution from grover results
                output = self.get_sol(grover_output)
                schedule = self.convert_solution_int(output,n)
                cmax = self.calculate_makespan(pm,np.array(schedule))
                algo_tracking.append(
                    {
                        "schedule" : schedule,
                        "cmax" : cmax,
                        "rotation" : r,
                        "iteration" : it,
                    }
                )
                if cmax < optimum_value and self.feasibility_check(schedule):
                    optimum_value = cmax
                    optimum_permu = schedule
                    impovement = True
                    upperBound = cmax
                    # solution trace 
                    trace.append((optimum_permu,optimum_value))
                else:
                    m = int(np.ceil(min(m*8/7,2**(n*N/2))))   
                    if schedule not in schedule_measurement :
                        schedule_measurement.append(schedule)
                    if (nb_no_improvement >= self.num_iteration 
                        or r >= max_r
                        or len(schedule_measurement) == num_solutions):
                        impovement = True
                        optimum_found = True
                    print(nb_no_improvement >= self.num_iteration,r >= max_r,len(schedule_measurement) == num_solutions)

        return (trace,algo_tracking)
                           
    def qubits_cmj_Estimation(self) ->int :
        """
            Estimate the required number of qubits for the circuit
         """
        n = self.num_jobs
        m = self.num_machines
        return math.ceil(math.log2(sum(self.process_time[i][j] for i in range(m) for j in range(n) )))

    def calculate_makespan(self,a, seq):
        a = np.transpose(a)
        # Order the jobs (rows) in order of the sequence
        a = a[seq]

        b = np.zeros(a.shape)
        jobs = a.shape[0]
        macs = a.shape[1]

        b[0, 0] = a[0, 0]
        # Build first row
        for i in range(1, macs):
            b[0, i] = a[0, i] + b[0, i - 1]
        # Build first column
        for i in range(1, jobs):
            b[i, 0] = a[i, 0] + b[i - 1, 0]
        # Build the rest
        for i in range(1, jobs):
            for j in range(1, macs):
                b[i, j] = a[i, j] + (b[i - 1, j] if b[i - 1, j] > b[i, j - 1] else b[i, j - 1])

        return int(b[-1, -1])
 
    
    def convert_solution_int(self,bin_s,n):
        " Return converted solution "
        s = []
        for i in range(len(bin_s)//n):
            s.append(int(bin_s[i*n : i*n + n],2))
        return s
    
    def get_sol(self,a):  
        return algorithm_globals.random.choice(list(a.keys()), 1, p=list(a.values()))[0]

    def feasibility_check(self,a : List):
        return len(set(a)) == len(a)







In [3]:

IBMQ.load_account()
       
my_provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
        
device = my_provider.get_backend("simulator_mps") 
        
quantum_instance = QuantumInstance(device, shots =1000,skip_qobj_validation = False)
solver = GroverOptimizerFSP(2,2,[[4,2,2,1],[2,3,1,3]],10,quantum_instance,20)
r = solver.solve()
r

1
startGrover
{'00000000': 0.001, '01000000': 0.001, '11000000': 0.002, '00100000': 0.001, '10100000': 0.001, '01100000': 0.001, '11100000': 0.001, '00010000': 0.002, '11010000': 0.003, '00110000': 0.002, '01110000': 0.003, '11110000': 0.003, '00001000': 0.001, '10001000': 0.003, '01001000': 0.001, '11001000': 0.001, '00101000': 0.001, '00011000': 0.001, '10011000': 0.001, '01011000': 0.002, '11011000': 0.027, '00111000': 0.004, '10111000': 0.001, '01111000': 0.032, '11111000': 0.003, '10000100': 0.002, '11000100': 0.003, '00100100': 0.001, '10100100': 0.002, '11100100': 0.023, '00010100': 0.003, '10010100': 0.006, '01010100': 0.002, '11010100': 0.001, '10110100': 0.029, '01110100': 0.001, '11110100': 0.002, '00001100': 0.001, '10001100': 0.001, '01001100': 0.004, '11001100': 0.002, '10101100': 0.003, '01101100': 0.022, '11101100': 0.002, '10011100': 0.022, '01011100': 0.002, '11011100': 0.001, '00111100': 0.001, '10111100': 0.001, '11111100': 0.001, '00000010': 0.001, '10000010': 0.00

In [ ]:

def convert_solution_int(bin_s,n):
    " Return converted solution "
    s = []
    for i in range(len(bin_s)//n):
        s.append(int(bin_s[i*n : i*n + n],2))
    return s

print(convert_solution_int("0111100100101011",4))

a = {'00000000': 0.001, '01000000': 0.004, '11000000': 0.003, '00100000': 0.001, '10100000': 0.002,
 '11100000': 0.001, '10010000': 0.004, '01010000': 0.003, '11010000': 0.001, '00110000': 0.002, 
 '10110000': 0.003, '01110000': 0.002, '11110000': 0.002, '00001000': 0.005, '10001000': 0.001, 
 '01001000': 0.006, '11001000': 0.004, '10101000': 0.001, '01101000': 0.002, '11101000': 0.002, 
 '00011000': 0.001, '10011000': 0.002, '01011000': 0.002, '11011000': 0.001, '00111000': 0.001, 
 '10111000': 0.001, '11111000': 0.002, '00000100': 0.003, '01000100': 0.002, '00100100': 0.002, 
 '10100100': 0.002, '01100100': 0.001, '11100100': 0.002, '00010100': 0.002, '10010100': 0.002, 
 '01010100': 0.001, '10110100': 0.001, '01110100': 0.002, '00001100': 0.005, '10001100': 0.002, 
 '01001100': 0.002, '11001100': 0.001, '00101100': 0.005, '01101100': 0.002, '11101100': 0.002, 
 '00011100': 0.001, '10011100': 0.001, '01011100': 0.002, '11011100': 0.001, '00111100': 0.001, 
 '01111100': 0.002, '00000010': 0.002, '10000010': 0.003, '01000010': 0.002, '11000010': 0.001, 
 '00100010': 0.001, '10100010': 0.001, '01100010': 0.002, '11100010': 0.003, '00010010': 0.004, 
 '10010010': 0.003, '01010010': 0.002, '00110010': 0.001, '10110010': 0.001, '11110010': 0.001, 
 '00001010': 0.002, '10001010': 0.003, '01001010': 0.002, '11001010': 0.002, '00101010': 0.002, 
 '10101010': 0.002, '11101010': 0.001, '00011010': 0.001, '10011010': 0.001, '01011010': 0.003, 
 '11011010': 0.003, '00111010': 0.002, '10111010': 0.003, '01111010': 0.002, '11111010': 0.001, 
 '00000110': 0.001, '10000110': 0.001, '11000110': 0.003, '00100110': 0.003, '10100110': 0.003, 
 '01100110': 0.001, '11100110': 0.003, '00010110': 0.003, '10010110': 0.003, '11010110': 0.003, 
 '00110110': 0.002, '10110110': 0.001, '01110110': 0.001, '00001110': 0.003, '10001110': 0.005, 
 '01001110': 0.001, '11001110': 0.004, '00101110': 0.002, '10101110': 0.002, '01101110': 0.002, 
 '00011110': 0.002, '10011110': 0.001, '11011110': 0.001, '00111110': 0.001, '10111110': 0.002, 
 '01111110': 0.001, '10000001': 0.002, '01000001': 0.002, '11000001': 0.002, '00100001': 0.004, 
 '10100001': 0.002, '01100001': 0.002, '00010001': 0.001, '10010001': 0.001, '01010001': 0.002,
  '00110001': 0.001, '10110001': 0.001, '01110001': 0.001, '11110001': 0.002, '00001001': 0.002, 
  '01001001': 0.002, '00101001': 0.001, '10101001': 0.004, '01101001': 0.003, '11101001': 0.002, 
  '00011001': 0.001, '11011001': 0.002, '00111001': 0.002, '10111001': 0.005, '01111001': 0.001, 
  '11111001': 0.004, '00000101': 0.002, '10000101': 0.002, '11000101': 0.001, '00100101': 0.001, 
  '10100101': 0.005, '01100101': 0.002, '00010101': 0.004, '10010101': 0.002, '01010101': 0.002, 
  '11010101': 0.001, '00110101': 0.001, '10110101': 0.001, '01110101': 0.004, '11110101': 0.001, 
  '00001101': 0.003, '10001101': 0.141, '01001101': 0.001, '11001101': 0.002, '00101101': 0.002, '10101101': 0.004, '01101101': 0.001, '11101101': 0.002, '01011101': 0.001, '11011101': 0.002, '00111101': 0.003, '10111101': 0.003, '01111101': 0.002, '11111101': 0.001, '00000011': 0.001, '10000011': 0.001, '11000011': 0.002, '00100011': 0.001, '10100011': 0.001, '01100011': 0.145, '11100011': 0.001, '00010011': 0.001, '10010011': 0.137, '01010011': 0.001, '11010011': 0.001, '10110011': 0.001, '01110011': 0.001, '00001011': 0.001, '10001011': 0.005, '01001011': 0.003, '01101011': 0.002, '11101011': 0.001, '00011011': 0.004, '10011011': 0.001, '01011011': 0.003, '11011011': 0.002, '00111011': 0.002, '10111011': 0.001, '01111011': 0.002, '11111011': 0.001, '10000111': 0.153, '01000111': 0.002, '11000111': 0.005, '00100111': 0.005, '10100111': 0.001, '01100111': 0.003, '11100111': 0.002, '00010111': 0.003, '01010111': 0.003, '11010111': 0.002, '00110111': 0.001, '10110111': 0.001, '10001111': 0.003, '01001111': 0.001, '11001111': 0.004, '00101111': 0.001, '10101111': 0.001, '01101111': 0.002, '11101111': 0.003, '00011111': 0.002, '10011111': 0.003, '11011111': 0.003, '00111111': 0.001, '10111111': 0.001, '01111111': 0.002, '11111111': 0.002}

def get_sol(a):  
    return algorithm_globals.random.choice(list(a.keys()), 1, p=list(a.values()))[0]

def feasibility_check(a : List):
    return len(set(a)) == len(a)

b = get_sol(a)
print(convert_solution_int(b,2))
print(b,feasibility_check(convert_solution_int(b,2)))
